In [ ]:
!pip install streamlit

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.58s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!npm audit fix

npm ERR! code EAUDITNOPJSON
npm ERR! audit No package.json found: Cannot audit a project without a package.json

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-04-18T00_38_35_532Z-debug.log


In [ ]:
%%writefile app.py

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.232.251.135


In [ ]:
# Abre o arquivo
with open('app.py', 'r') as arquivo:
  arquivo.read()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.232.251.135:8501

npx: installed 22 in 2.312s
your url is: https://eighty-baboons-attack.loca.lt


#Código da aplicação

In [ ]:

import numpy     as np
import pandas    as pd
import streamlit as st

from datetime    import datetime
from PIL         import Image
from io          import BytesIO


# Imports
import pandas            as pd
import streamlit         as st
import numpy             as np

from datetime            import datetime
from PIL                 import Image
from io                  import BytesIO

@st.cache_data
def convert_df(df):
    return df.to_csv(index=False).encode('utf-8')

# Funço para converter o df para excel
@st.cache_data
def to_excel(df):
    output = BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    writer.close()
    processed_data = output.getvalue()
    return processed_data


# Criando os segmentos
def recencia_class(x, r, q_dict):
    """Classifica como melhor o menor quartil
       x = valor da linha,
       r = recencia,
       q_dict = quartil dicionario
    """
    if x <= q_dict[r][0.25]:
        return 'A'
    elif x <= q_dict[r][0.50]:
        return 'B'
    elif x <= q_dict[r][0.75]:
        return 'C'
    else:
        return 'D'

def freq_val_class(x, fv, q_dict):
    """Classifica como melhor o maior quartil
       x = valor da linha,
       fv = frequencia ou valor,
       q_dict = quartil dicionario
    """
    if x <= q_dict[fv][0.25]:
        return 'D'
    elif x <= q_dict[fv][0.50]:
        return 'C'
    elif x <= q_dict[fv][0.75]:
        return 'B'
    else:
        return 'A'

# Funço principal da aplicaço
def main():
    # Configuraço inicial da pÃ¡gina da aplicaço
    st.set_page_config(
        page_title="EBAC | Móddulo 31 | Streamlit V | Exerci­cio 1",
        page_icon='https://media.licdn.com/dms/image/D4D03AQGHn0eRwZy0wA/profile-displayphoto-shrink_800_800/0/1701657426971?e=1718841600&v=beta&t=pft3rYZtma1hYrfAJBLgvihX06zyFE4_qk_UF-WilCg',
        layout="wide",
        initial_sidebar_state="expanded",
    )

    # TíTULO
    st.markdown(
        """
    <div style="text-align:center">
        <a href="https://github.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados">
            <img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/ebac_logo-data_science.png" alt="ebac_logo-data_science" width=100%>
        </a>
    </div>

    ---

    <!-- # **Profissão: Cientista de Dados** -->
    ### **Módulo 31** | Streamlit V | Exerci­cio 1

    **Aluno:** [Victor Chicati Sobradiel](https://www.linkedin.com/in/victorchicati/)<br>
    **Data:** 16/04/2024.

    ---
    """,
        unsafe_allow_html=True,
    )

    # Tí­tulo principal da aplicaço
    st.write("""# RFV

    RFV significa recência, frequência, valor e é utilizado para segmentaço de clientes baseado no comportamento
    de compras dos clientes e agrupa eles em clusters parecidos. Utilizando esse tipo de agrupamento podemos realizar
    ações de marketing e CRM melhores direcionadas, ajudando assim na personalizaço do conteúdo e até a retenço de clientes.

    Para cada cliente é preciso calcular cada uma das componentes abaixo:

    - Recência (R): Quantidade de dias desde a última compra.
    - Frequência (F): Quantidade total de compras no perÃ­odo.
    - Valor (V): Total de dinheiro gasto nas compras do perÃ­odo.

    E é isso que iremos fazer abaixo.
    """)
    st.markdown("---")

    # Apresenta a imagem na barra lateral da aplicaço
    st.sidebar.image('https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/newebac_logo_black_half.png')

    # BotÃ£o para carregar arquivo na aplicaço
    st.sidebar.write("## Suba o arquivo")
    data_file_1 = st.sidebar.file_uploader("Bank marketing data", type = ['csv','xlsx'])

    if data_file_1 is None:
        if st.sidebar.button('Carregar Arquivo Demonstrativo',
                            help='./input/dados.csv',
                            use_container_width=True,
                            key='uploadDemoButton',
                            disabled=st.session_state.get('uploadDemoButton', False)):
            data_file_1 = './input/dados.csv'
            st.sidebar.button('Limpar e Reiniciar', use_container_width=True)


    # Verifica se hÃ¡ conteúdo carregado na aplicaço
    if (data_file_1 is not None):
        df_compras = pd.read_csv(data_file_1, infer_datetime_format=True, parse_dates=['DiaCompra'])

        st.write('## Recência (R)')


        dia_atual = df_compras['DiaCompra'].max()
        st.write('Dia mÃ¡ximo na base de dados: ', dia_atual)

        st.write('Quantos dias faz que o cliente fez a sua última compra?')

        df_recencia = df_compras.groupby(by='ID_cliente', as_index=False)['DiaCompra'].max()
        df_recencia.columns = ['ID_cliente','DiaUltimaCompra']
        df_recencia['Recencia'] = df_recencia['DiaUltimaCompra'].apply(lambda x: (dia_atual - x).days)
        st.write(df_recencia.head())

        df_recencia.drop('DiaUltimaCompra', axis=1, inplace=True)

        st.write('## Frequência (F)')
        st.write('Quantas vezes cada cliente comprou com a gente?')
        df_frequencia = df_compras[['ID_cliente','CodigoCompra']].groupby('ID_cliente').count().reset_index()
        df_frequencia.columns = ['ID_cliente','Frequencia']
        st.write(df_frequencia.head())

        st.write('## Valor (V)')
        st.write('Quanto que cada cliente gastou no periodo?')
        df_valor = df_compras[['ID_cliente','ValorTotal']].groupby('ID_cliente').sum().reset_index()
        df_valor.columns = ['ID_cliente','Valor']
        st.write(df_valor.head())


        st.write('## Tabela RFV final')
        df_RF = df_recencia.merge(df_frequencia, on='ID_cliente')
        df_RFV = df_RF.merge(df_valor, on='ID_cliente')
        df_RFV.set_index('ID_cliente', inplace=True)
        st.write(df_RFV.head())

        st.write('## Segmentaço utilizando o RFV')
        st.write("Um jeito de segmentar os clientes é criando quartis para cada componente do RFV, sendo que o melhor quartil é chamado de 'A', o segundo melhor quartil de 'B', o terceiro melhor de 'C' e o pior de 'D'. O melhor e o pior depende da componente. Po exemplo, quanto menor a recência melhor é o cliente (pois ele comprou com a gente tem pouco tempo) logo o menor quartil seria classificado como 'A', jÃ¡ pra componente frêquencia a lÃ³gica se inverte, ou seja, quanto maior a frêquencia do cliente comprar com a gente, melhor ele/a é, logo, o maior quartil recebe a letra 'A'.")
        st.write('Se a gente tiver interessado em mais ou menos classes, basta a gente aumentar ou diminuir o número de quantils pra cada componente.')

        st.write('Quartis para o RFV')
        quartis = df_RFV.quantile(q=[0.25,0.5,0.75])
        st.write(quartis)

        st.write('Tabela apÃ³s a criaço dos grupos')
        df_RFV['R_quartil'] = df_RFV['Recencia'].apply(recencia_class,
                                                        args=('Recencia', quartis))
        df_RFV['F_quartil'] = df_RFV['Frequencia'].apply(freq_val_class,
                                                        args=('Frequencia', quartis))
        df_RFV['V_quartil'] = df_RFV['Valor'].apply(freq_val_class,
                                                    args=('Valor', quartis))
        df_RFV['RFV_Score'] = (df_RFV.R_quartil
                            + df_RFV.F_quartil
                            + df_RFV.V_quartil)
        st.write(df_RFV.head())

        st.write('Quantidade de clientes por grupos')
        st.write(df_RFV['RFV_Score'].value_counts())

        st.write('#### Clientes com menor recência, maior frequência e maior valor gasto')
        st.write(df_RFV[df_RFV['RFV_Score']=='AAA'].sort_values('Valor', ascending=False).head(10))

        st.write('### Ações de marketing/CRM')

        dict_acoes = {'AAA': 'Enviar cupons de desconto, Pedir para indicar nosso produto pra algum amigo, Ao lanÃ§ar um novo produto enviar amostras grÃ¡tis pra esses.',
        'DDD': 'Churn! clientes que gastaram bem pouco e fizeram poucas compras, fazer nada',
        'DAA': 'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar',
        'CAA': 'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar'
        }

        df_RFV['acoes de marketing/crm'] = df_RFV['RFV_Score'].map(dict_acoes)
        st.write(df_RFV.head())


        # df_RFV.to_excel('./auxiliar/output/RFV_.xlsx')
        df_xlsx = to_excel(df_RFV)
        st.download_button(label='Download',
                           data=df_xlsx,
                           file_name= 'RFV_.xlsx')

        st.write('Quantidade de clientes por tipo de aço')
        st.write(df_RFV['acoes de marketing/crm'].value_counts(dropna=False))

if __name__ == '__main__':
	main()
